In [1]:
import os
import pandas as pd
import numpy as np
import json
import google.generativeai as genai
from servicios.pydantic_model import ProductDimensions
from dotenv import load_dotenv
from tavily import TavilyClient

/home/mariano/anaconda3/envs/llmops/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
load_dotenv()
tavily_api_key = os.getenv("TAVILY_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")


In [3]:
df = pd.read_csv("files/test.csv")
df_llm = df.head(7).copy()

In [5]:
total_facturado = df_llm['tarifa'].sum()
total_facturado

np.int64(66050)

np.int64(67050)

In [12]:
tavily_client = TavilyClient(api_key=tavily_api_key)
genai.configure(api_key=google_api_key)

In [9]:
def extraer_datos_con_gemini(contexto: str, nombre_producto: str) -> dict:
    """Usa Gemini para extraer las dimensiones del texto de búsqueda."""
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    prompt = f"""
    Basado en el siguiente contexto de búsqueda para el producto "{nombre_producto}", 
    extrae el alto, ancho, largo y peso.
    
    - Si encuentras las dimensiones exactas, úsalas.
    - Si no las encuentras, busca dimensiones de productos muy similares mencionados en el contexto.
    - Devuelve los valores como números flotantes (float).
    - Tu respuesta DEBE ser únicamente un objeto JSON con las claves "alto", "ancho", "largo", "peso".
    - En la clave "fuente" indica de que pagina web o fuente obtuviste la información.
    - Si no encuentras la fuente, pon "desconocida".
    - Si no puedes determinar alguno de los valores, intenta predecirlo basado en productos similares. determina que tipo de producto es y busca dimensiones típicas.
    - Selecciona la opción más representativa y confiable, utiliza las medidas que a veces suelen estar en el nombre de la publicación o en las imagenes. 
    - Sé especialmente ágil y eficiente al estimar medidas y **sobretodo** peso físico.

    Contexto:
    ---
    {contexto}
    ---
    """
    
    try:
        response = model.generate_content(prompt)
        # Limpiar la respuesta para asegurar que sea un JSON válido
        json_text = response.text.strip().replace("```json", "").replace("```", "")
        return json.loads(json_text)
    except (json.JSONDecodeError, Exception) as e:
        print(f"  - Error al procesar la respuesta de Gemini: {e}")
        return {}


def buscar_dimensiones_producto(nombre_producto: str) -> dict:
    """Usa Tavily para buscar y Gemini para extraer las dimensiones."""
    print(f"\n🔎 Buscando con Tavily: '{nombre_producto}'...")
    
    try:
        # Búsqueda avanzada con Tavily
        search_context = tavily_client.search(
            query=f'dimensions (height, width, length, weight) for product "{nombre_producto}"',
            search_depth="advanced" # Búsqueda más profunda
        )
        
        # Combinamos los resultados para dárselos a Gemini
        contexto_combinado = "\n".join([str(res) for res in search_context['results']])
        
        if not contexto_combinado:
            print("  - Tavily no devolvió resultados.")
            return {}
            
        print("  - Resultados de Tavily obtenidos. Extrayendo con Gemini...")
        datos_extraidos = extraer_datos_con_gemini(contexto_combinado, nombre_producto)
        
        if datos_extraidos:
            print("  - ✅ ¡Extracción con Gemini exitosa!")
            datos_extraidos['fuente'] = "Tavily + Gemini"
            return datos_extraidos
        else:
            print("  - 🟡 Gemini no pudo extraer los datos del contexto.")
            return {}

    except Exception as e:
        print(f"  - ❌ Error durante la búsqueda con Tavily: {e}")
        return {}

In [ ]:
productos = []
for name in df_llm['name']:
    print(f"\nProcesando producto: {name}")
    dimensiones = buscar_dimensiones_producto(name)
    if dimensiones:
        try:
            producto = ProductDimensions(**dimensiones)
            print(f"  - Dimensiones extraídas: {producto}")
            productos.append(producto)
        except Exception as e:
            print(f"  - Error al validar las dimensiones con Pydantic: {e}")
    else:
        print("  - No se pudieron extraer dimensiones para este producto.")


Procesando producto: SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards

🔎 Buscando con Tavily: 'SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...
  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=5.0 ancho=20.0 largo=110.0 peso=0.5 fuente='Tavily + Gemini'

Procesando producto: ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms

🔎 Buscando con Tavily: 'ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...
  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=10.0 ancho=0.1 largo=35.0 peso=47.25 fuente='Tavily + Gemini'

Procesando producto: AUTOWORLD MAX BOLERO FRONT GRILL WITH CROME CENTRE BLACK FOR MAHINDRA BOLERO TYPE-3

🔎 Buscando con Tavily: 'AUTOWORLD MAX BOLERO FRONT GRILL WIT

In [39]:
producto_encontrado = productos[0]

alto_valor = producto_encontrado.alto
ancho_valor = producto_encontrado.ancho
largo_valor = producto_encontrado.largo
peso_valor = producto_encontrado.peso # En gramos, según el ejemplo

print(f"Alto: {alto_valor} cm")
print(f"Ancho: {ancho_valor} cm")
print(f"Largo: {largo_valor} cm")
print(f"Peso: {peso_valor} g")

Alto: 10.0 cm
Ancho: 5.0 cm
Largo: 10.0 cm
Peso: 270.0 g


In [ ]:
from servicios.consulta_invoices import trae_invoices
from servicios.consulta_tarificacion import trae_tarifas

df = trae_invoices(202511)
tarifas = trae_tarifas()


RMKeyView(['id', 'periodo', 'proveedor', 'track_code', 'ambito', 'tipo_servicio', 'name', 'main_category', 'sub_category', 'category', 'alto', 'ancho', 'largo', 'peso_aforado', 'peso_fisico', 'peso_facturable', 'tarifa'])
[(10846, 202511, 'BoxFast', 'BFX42754874321X', 2, '24hs', 'SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards', 'car & motorbike', 'Car Parts', 'Car_Parts', Decimal('32.00'), Decimal('18.00'), Decimal('69.00'), Decimal('13.25'), Decimal('12.20'), Decimal('13.25'), Decimal('5600.00')), (10847, 202511, 'BoxFast', 'BFX45088662684X', 2, '24hs', 'ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms', 'car & motorbike', 'Car Parts', 'Car_Parts', Decimal('67.00'), Decimal('50.00'), Decimal('30.00'), Decimal('33.50'), Decimal('19.77'), Decimal('33.50'), Decimal('10100.00')), (10848, 202511, 'BoxFast', 'BFX45832574342X', 2, '72hs', 'AUTOWORLD MAX BOLERO FRONT GRILL WITH CROME CENTRE BLACK FOR MAHINDRA BOLERO 

In [17]:
df.head(1)

,id,periodo,proveedor,track_code,ambito,tipo_servicio,name,main_category,sub_category,category,alto,ancho,largo,peso_aforado,peso_fisico,peso_facturable,tarifa
0,10846,202511,BoxFast,BFX42754874321X,2,24hs,SDR Rear Bumper Step footstep Trim For Toyota ...,car & motorbike,Car Parts,Car_Parts,32.00,18.00,69.00,13.25,12.20,13.25,5600.00


In [ ]:
tarifas.head(1)

,id,proveedor,fecha_inicio,fecha_fin,ambito,tipo_de_servicio,rango_desde,rango_hasta,tarifa
0,1,BoxFast,2025-07-01,2025-12-31,1,24hs,0,5,2700.00


In [19]:
len(tarifas)

99

In [20]:
tarifas.columns

Index(['id', 'proveedor', 'fecha_inicio', 'fecha_fin', 'ambito',
       'tipo_de_servicio', 'rango_desde', 'rango_hasta', 'tarifa'],
      dtype='object')

In [ ]:
productos = []
for row in df.itertuples():
    print(f"\nProcesando producto: {row.name}")
    dimensiones = buscar_dimensiones_producto(row.name)
    if dimensiones:
        try:
            producto = ProductDimensions(**dimensiones)
            print(f"  - Dimensiones extraídas: {producto}")
            id = row.id
            ambito = row.ambito
            operacion = row.tipo_servicio
            alto_valor = producto.alto
            ancho_valor = producto.ancho
            largo_valor = producto.largo
            peso_valor = producto.peso
            try:
                peso_aforado = ancho_valor * largo_valor * alto_valor / 4000 # Peso aforado en kg
            except Exception as e:
                print(f"  - Error al calcular el peso aforado: {e}")
                peso_aforado = 0
            try:    
                peso_facturable = max(peso_valor, peso_aforado) # Peso facturable en kg
            except Exception as e:
                print(f"  - Error al calcular el peso facturable: {e}")
                peso_facturable = 0
            tarifa_real = tarifas[
                (tarifas['ambito'] == ambito) & 
                (tarifas['tipo_de_servicio'] == operacion) & 
                (tarifas['rango_desde'] <= peso_facturable) & 
                (tarifas['rango_hasta'] >= peso_facturable)
            ]
            datos = {
                "id": id,
                "ambito": ambito,
                "tipo_servicio": operacion,
                "alto_cm": alto_valor,
                "ancho_cm": ancho_valor,
                "largo_cm": largo_valor,
                "peso_g": peso_valor,
                "peso_aforado_kg": peso_aforado,
                "peso_facturable_kg": peso_facturable,
                "tarifa_real": tarifa_real['tarifa'].values[0] if not tarifa_real.empty else np.nan
            }
            print(datos)
            productos.append(datos)
            
        except Exception as e:
            print(f"  - Error al validar las dimensiones con Pydantic: {e}")
    else:
        print("  - No se pudieron extraer dimensiones para este producto.")


Procesando producto: SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards

🔎 Buscando con Tavily: 'SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...
  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=3.0 ancho=12.0 largo=90.0 peso=0.5 fuente='Tavily + Gemini'
{'id': 10846, 'ambito': 2, 'tipo_servicio': '24hs', 'alto_cm': 3.0, 'ancho_cm': 12.0, 'largo_cm': 90.0, 'peso_g': 0.5, 'peso_aforado_kg': 0.81, 'peso_facturable_kg': 0.81, 'tarifa_real': Empty DataFrame
Columns: [id, proveedor, fecha_inicio, fecha_fin, ambito, tipo_de_servicio, rango_desde, rango_hasta, tarifa]
Index: []}

Procesando producto: ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms

🔎 Buscando con Tavily: 'ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms'...
  - Resultados de Tavily obtenidos. E

KeyboardInterrupt: 